Initializing Libraries and Global Variables

In [ ]:
import pickle
import pandas as pd
import psycopg2

In [12]:
path = r'D:\\Projects\\ipums\\garrard_test\\'
cbkname = r'ky_1870_cbk.pickle'
prefix = 'cn70_'
cbkfile = path + cbkname

Pickle functions

In [13]:
def load_pickle(filename, objectname):
        with open(path+filename, "rb") as infile:
            objectname = pickle.load(infile)
        print('Loaded from:', filename)

def save_pickle(filename, objectname):
    with open(path+filename, "wb") as outfile:
        pickle.dump(objectname, outfile)
    print('Saved to:',filename)

Loading the codebook and creating the column list

In [14]:

cbk = pd.read_pickle(cbkfile)
cbk.columns
x = cbk['length'].astype('str')
cbk['column_name'] = prefix+cbk['variable'].str.lower() + ' char('+x+')'
column_list = cbk['column_name'].tolist()
print('column_list:\n',column_list[0:10])

column_list:
 ['cn70_year char(4)', 'cn70_serial char(8)', 'cn70_numprec char(2)', 'cn70_subsamp char(2)', 'cn70_numperhh char(4)', 'cn70_hhtype char(1)', 'cn70_region char(2)', 'cn70_stateicp char(2)', 'cn70_statefip char(2)', 'cn70_countyicp char(4)']


Database Functions (psycopg2)

In [15]:
def open_database(database):
    conn = psycopg2.connect(dbname=database, user='postgres', password = '3701', host='localhost', port='5432')
    return conn

def create_cursor():
    cursor = conn.cursor()
    return cursor

def set_census_schema():
    cur.execute("SET SCHEMA 'census'")

def create_table(tablename, column_list):
    columns_text = '(' + 'id serial PRIMARY KEY, ' + ' '.join(column_list) + ');'
    print(columns)
    cur.execute('DROP TABLE IF EXISTS ' + tablename + ';')
    cur.execute('CREATE TABLE ' + tablename + ' ' + columns_text)
    cur.execute('SELECT * FROM ' + tablename + ';') #to confirm table is created with correct column names

def get_column_count(tablename, columns):
    qry_cc = '''
    select count(*) from information_schema.columns
    where table_name='<table_name>’;
    '''
    col_ct_sql = cur.execute(qry_cc)
    col_ct_list = len(columns)+1
    print('columns in table:',col_ct_sql,'\n','columns in list:',col_ct_list)
    
def close_database():
    conn.commit() #insert rollback logic
    conn.close()
      

Open the Database

In [16]:
conn = open_database('test')
cur = create_cursor()
set_census_schema()

Create the Census table in postgresSQL

In [17]:
#table_name = 'census_1870'
#create_table(table_name,col_list)
#get_column_counts('table_name,col_list)
